In [74]:
# import project

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
import ollama


In [75]:
MODEL = "llama3.2"

# class for the website
class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)
        

In [77]:
system_prompt = "You are an assistant that analyzes the offers of two websites \
and provides a short summary for both sites, ignoring text that might be navigation related. \
Respond with the offers from both sites."

In [78]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website1, website2):
    user_prompt = f"You are looking at two websites titled {website1.title}, {website2.title}"

    # 1st Website 
    user_prompt += f"### website one: {website1.title}\n"
    user_prompt += "The contents of this website are as follows;"
    user_prompt += "please provide a short summary of this website in markdown."
    user_prompt += "If it includes news or annoucements, then summarize these too.\n\n"
    user_prompt += website1.text + "\n\n"

    # 2nd website
    user_prompt += f"### Website two: {website2.title}\n"
    user_prompt += "The contents of this website are as follow;"
    user_prompt += "please provide a short summary of this websire in markdown."
    user_prompt += "If it includes news or annoucements, then summarize these too.\n\n"
    user_prompt += website2.text + "\n\n"

    
    # Compare the websites
    user_prompt += "Finally, compare the offers, announcements, or any relevant information "
    user_prompt += "From both websites and highlight key differences and provide which option is better.\n"

    return user_prompt
    


In [79]:
# See how this function creates exactly the format above

def messages_for(website1, website2):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website1, website2)}
    ]

In [80]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url1, url2):
    website1 = Website(url1)
    website2 = Website(url2)
    response = ollama.chat(
        model = "llama3.2",
        messages = messages_for(website1, website2)
    )
    return response['message']['content']

In [81]:
# A function to display this nicely in the Jupyter output, using markdown

def displaySummary(url1, url2):
    # Generate the combined summary for both websites
    combined_summary = summarize(url1, url2)
    
    # Display the combined summary in Markdown format
    display(Markdown(combined_summary))

In [82]:
displaySummary("https://www.t-mobile.com/cell-phone/apple-iphone-16-pro-max",
    "https://www.att.com/buy/phones/apple-iphone-16-pro-max.html"
)

After comparing the two websites from AT&T and Apple, here are the key differences:

**Device Specifications**

* AT&T:
	+ iPhone 16 Pro Max: 6.7 inches, 512GB storage
	+ Price: $1,099 (plus taxes and fees)
* Apple:
	+ iPhone 16 Pro Max: 6.7 inches, 512GB storage
	+ Price: Not specified on the website

**Monthly Plans**

* AT&T:
	+ Unlimited Starter Plan: $60/month (plus taxes and fees)
	+ Unlimited Premium Plan: $85/month (plus taxes and fees)
* Apple:
	+ No monthly plan information available on the website. However, it's likely that the iPhone 16 Pro Max will be eligible for a variety of plans, including AT&T's unlimited plans.

**Promotions**

* AT&T:
	+ Free upgrade to an iPhone 16 Pro Max every year
	+ $200 trade-in credit towards any new iPhone
* Apple:
	+ No promotions available on the website. However, it's likely that Apple will offer some promotional pricing or discounts for the iPhone 16 Pro Max at a later date.

**Additional Features**

* AT&T:
	+ Wi-Fi Calling and FaceTime video calling over cellular or Wi-Fi
	+ HD Voice capable
	+ Face ID with TrueDepth camera
* Apple:
	+ Face ID with TrueDepth camera
	+ Water-resistant (IP68 rating)
	+ Fast charging (up to 20W)

**Key Differences**

* AT&T offers a free upgrade to an iPhone every year, while Apple does not offer any promotional pricing or upgrades.
* AT&T's Unlimited Premium Plan costs $85/month, while Apple does not specify the monthly plan cost for the iPhone 16 Pro Max.

**Recommendation**

Based on the information available on the websites, it appears that AT&T is offering a more competitive price point for the iPhone 16 Pro Max. Additionally, their free upgrade offer every year may be attractive to customers who want to stay up-to-date with the latest Apple devices. However, if you're invested in the Apple ecosystem and prefer to purchase directly from Apple, they may still be the better option.

**Conclusion**

If you prioritize price and flexibility, AT&T's offering for the iPhone 16 Pro Max may be the better choice. However, if you're committed to purchasing directly from Apple and value their ecosystem, you may want to wait until a promotional offer is available or consider visiting an Apple Store for more information.